In [16]:

using JLD
using DSP # For conv function in Psi
using Dates

include("Model_KSE.jl")
include("Model_Reduction_matpred.jl")


vector_wiener_filter_fft (generic function with 4 methods)

In [49]:
gen = 1
T = 1000 # Length (in seconds) of time of run
T_disc = 400 # Length (in seconds) of time discarded
P = 32π  # Period
N = 128  # Number of fourier modes used
h = 1e-3 # Timestep
g = x -> cos(π*x/16)*(1 + sin.(π*x/16))
q = 2π/P*(0:N-1)
obs_gap = 10
d = 5 # No. of lowest modes taken in reduced model
M_out = 1024 # No
short = false
loadsol = false
loadwf = false

false

In [50]:
uu, vv, tt =  my_KSE_solver(T,
       T_disc  = T_disc,
       P = P,
       N = N,
       h = h,
       g = g,
       n_gap = obs_gap);

In [46]:
V_obs = vv[2:d+1,1:end];
size(V_obs)

(5, 6001)

In [51]:
function InvBurgRK4_1step(x)
  lx = length(x)
  function F(x)
      𝑥 = [conj(reverse(x, dims = 1));0; x]
      conv(𝑥,𝑥)[2*lx+2:3*lx+1]
  end

  Δt = h*obs_gap

  k1 = F(x)
  k2 = F(x .+ Δt*k1/2)
  k3 = F(x .+ Δt*k2/2)
  k4 = F(x .+ Δt*k3)
  A =  @. x + Δt/6*(k1 + 2k2 + 2k3 + k4)
end

function Inertialman_part(x)
  lx = length(x)
  𝑥(j) = ( j <= lx ? x[j] : im*sum(x[l]*x[j-l] for l = j-lx:lx) )

  L = complex(zeros(lx,lx))
  for j = 1:lx
     for k = 1:lx
        L[k,j] = 𝑥(j+lx)*𝑥(j+lx-k)
     end
  end
  L
end

Psi(x) = [x InvBurgRK4_1step(x) Inertialman_part(x)]

Psi (generic function with 1 method)

In [57]:
s = V_obs[:,2]
Psi(s)[:,2:4]

5×3 Array{Complex{Float64},2}:
  -327.521+584.817im  -832892.0+1.55773e6im  1.40788e8-5.15451e7im
   3004.64-572.635im   649744.0-314651.0im   -110704.0-1.13757e6im
  -457.316-15.841im    251496.0+981836.0im   -236863.0+402604.0im
   1309.14+1375.23im  -580523.0-99220.5im    -487799.0-438318.0im
 0.0950815-356.648im  -276729.0+657974.0im    348145.0-154946.0im

In [29]:
x = randn(5) + im*randn(5)
Psi(x)

5×7 Array{Complex{Float64},2}:
 -0.397865-0.183089im   0.143206-0.256942im  …     1.24158-1.29256im
 -0.515415-0.339361im  -0.423915-0.606429im        2.14081-2.03557im
 -0.311734+0.939181im  -0.380782+0.797055im         1.7085-1.60516im
  -1.32259+1.35534im    -1.30063+1.36328im         1.40585+0.623473im
 -0.387858+0.675885im    -0.1929+0.651875im     0.00373091+0.473197im

In [48]:
@time h_wf = get_wf_matpred(V_obs,Psi, M_out = M_out,PI = false)

  4.147592 seconds (4.43 M allocations: 4.041 GiB, 11.03% gc time)


7×1024 Array{Float64,2}:
    -3.22451e18   1.86957e18   4.822e17    …  31500.0    11774.0   -9612.0
 -4268.91         1.47687e16  -4.81077e16      2221.25      63.25  -2907.38
    -4.27977e9    1.06375e17  -3.95339e16     -1166.25   -4736.75  -7716.75
    -9.31722e6    5.7058e16   -5.36235e16       319.906   1790.59   2564.38
    -6.83171e6    1.51938e17  -8.47734e16     -2218.5    -1130.75   -701.5
     9.36443e6    1.50114e17  -6.18386e16  …  -1431.0    -1473.75  -1429.0
     3.12129e6   -1.17386e17   3.22354e16       662.75    1719.5    2865.75

In [42]:
signal = V_obs

sig = signal[:,2:end]
d, steps = size(sig)
dp, nu = size(Psi(zeros(d,1)))
d == dp || error("first dimension of pred should equal dimension of sig.")

pred = complex(zeros(d,nu, steps))
for n = 1:steps
    pred[:,:,n] = Psi(signal[:,n])
end

In [43]:
pred

5×7×6000 Array{Complex{Float64},3}:
[:, :, 1] =
 58.2871-81.6176im  1.30205e19-1.18646e18im  …  2.64075e8-4.76066e7im
 23.3021+18.3148im  1.06899e19-6.93769e17im       7.637e7+2.31309e7im
 12.8744-7.97858im  8.95611e18-2.59334e18im     1.35003e8+5.62032e7im
 69.7378+6.75058im  6.09782e18-3.21991e18im     1.29872e8-2.98351e8im
   51.99-112.772im  6.30328e18-3.86674e18im     -519730.0-1.84303e6im

[:, :, 2] =
 58.5243-81.5989im  1.28679e19-1.22903e18im  …  2.45753e8-4.69717e7im
 23.0523+18.1014im   1.0558e19-7.59299e17im     7.34514e7+2.14958e7im
 12.9002-8.18472im  8.85255e18-2.62874e18im     1.25922e8+5.1851e7im
 69.8117+5.89249im  6.03484e18-3.27142e18im     1.23315e8-2.77491e8im
 50.8586-110.144im  6.17994e18-3.87277e18im     -481549.0-1.71946e6im

[:, :, 3] =
 58.7473-81.5576im  1.28419e19-1.29207e18im  …  2.27945e8-4.61846e7im
 22.8301+17.9138im  1.05338e19-8.48402e17im     7.07839e7+1.97058e7im
  12.972-8.4265im   8.84406e18-2.69892e18im     1.17346e8+4.78404e7im
 69.8738+4.90826i

In [20]:
steps = size(V_obs,2)
V_rm = [V_obs[:,1:M_out] complex(zeros(d,steps-M_out))]
nu = size(Psi(V_obs[:,1]),2)

# load presamples
PSI_past = complex(zeros(d,nu,steps))
for i=1:M_out
   PSI_past[:,:,i] = Psi(V_obs[:,i])
end

In [22]:
# Move forward without original data
print("Reduced Model Run Time: ")
@time for i = M_out+1:steps
   V_rm[:,i] = sum(PSI_past[:,:,i-k]*h_wf[:,k] for k = 1:M_out)
   isnan(V_rm[1,i]) && break
   PSI_past[:,:,i] = Psi(V_rm[:,i])
end

Reduced Model Run Time:   0.145737 seconds (356.36 k allocations: 18.157 MiB)


In [24]:
findall(x -> x == 0, V_rm[1,:])[1]

13